# DSAI HW1 Peak Load Forecasting
請根據台電歷史資料，預測未來七天的"電力尖峰負載"(MW)

In [4]:
import numpy as np
import pandas as pd
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dense

/Users/Solitude6060/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/Solitude6060/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## Data Preprocessing

Read data and parser - 
將CSV資料讀入後以逗號拆開，只取包含時間前4個feature

In [5]:
def readfile(path):
    file = open(path, "r+")
    f = file.readlines()
    return_tp_list = []
    test_list = []
    for i in f[1:366]:
        #print(i)
        day_list = i[:-1].split(",")
        temp_list = []
        for d in day_list[:2]:
            temp_list.append(d)
        for d in day_list[3:5]:
            temp_list.append(d)
        test_list.append(day_list[2])
        #print(temp_list)
        #print(len(temp_list))
        return_tp_list.append(temp_list)
    
    return return_tp_list, test_list

取2017和2018年的資料

In [6]:
tp_2017, target_2017_list = readfile('data/taipower_2017.csv')
tp_2018, target_2018_list = readfile('data/taipower_2018.csv')

#print(tp_2018)

Parser data ，取去年同一時間和上一週的資料

In [7]:
def sliceData(tp_data_2017, tp_data_2018):
    #index = 1
    con_list = []
    data_2017_list = []
    for t in range(len(tp_data_2017[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2017[t][1:]
        else:
            #if t is 0:
            con_list += tp_data_2017[t][1:]
            data_2017_list.append(con_list)
            con_list = []
    
    con_list = []
    data_2018_list = []
    for t in range(len(tp_data_2018[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2018[t][1:]
            #print(con_list)
        else:
            #if t is 0:
            con_list += tp_data_2018[t][1:]    
            data_2018_list.append(con_list)
            con_list = []
    
    #print("2017", data_2017_list[0])
    #print("2018", data_2018_list[0])
    train_list = []
    temp_list =[]
    for i in range(len(data_2017_list)):
        if i == 0:
            temp_list = data_2017_list[i] + data_2017_list[-1]
            #print("i = 0", temp_list)
        else:
            temp_list = data_2017_list[i] + data_2017_list[i]
            #print("i != 0", temp_list)
        train_list.append(temp_list)
    
    return train_list

In [8]:
train_X = sliceData(tp_2017, tp_2018)
#print(train_X[0])
#print(len(train_X[0]))

取得Training target，也就是training data中實際的尖峰負載預測值

In [9]:
def getTrainY(target_list):
    train_y_list = []
    con_list = []
    for i in range(len(target_list[:-1])):
        if (i+1) % 7 is not 0:
            con_list.append(target_list[i])
        else:
            con_list.append(target_list[i])
            train_y_list.append(con_list)
            con_list = []
    return train_y_list

In [10]:
train_Y = getTrainY(target_2018_list)
np_train_X = np.array(train_X)
np_train_Y = np.array(train_Y)

# Bulid Model
以Keras開發，用NN概念實現Linear regression
以多層Dense，最後一層output 1x7 向量代表整週的預測

In [29]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(x_shape[1], ), activation='linear'))
    model.add(Dense(units=64,activation='linear'))
    model.add(Dense(units=32,activation='linear'))
    model.add(Dense(units=7,activation='linear'))
    model.summary()
    return model

# Loss function
root_mean_squared_error

In [30]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [31]:
LinearModel = bulidModel(np_train_X.shape)
LinearModel.compile(loss=root_mean_squared_error, optimizer='rmsprop')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               11008     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 231       
Total params: 29,767
Trainable params: 29,767
Non-trainable params: 0
_________________________________________________________________


In [38]:
train_history = LinearModel.fit(np_train_X , np_train_Y , batch_size=64 , epochs=1000 , verbose=1, validation_split=0.2)

Train on 41 samples, validate on 11 samples
Epoch 1/1000
41/41 [==============================] - 0s 153us/step - loss: 2397.3533 - val_loss: 1408.7047
Epoch 2/1000
41/41 [==============================] - 0s 146us/step - loss: 2688.0845 - val_loss: 2888.3931
Epoch 3/1000
41/41 [==============================] - 0s 151us/step - loss: 2462.8882 - val_loss: 1884.4730
Epoch 4/1000
41/41 [==============================] - 0s 190us/step - loss: 2940.8008 - val_loss: 2399.5620
Epoch 5/1000
41/41 [==============================] - 0s 123us/step - loss: 2204.3215 - val_loss: 1349.9144
Epoch 6/1000
41/41 [==============================] - 0s 102us/step - loss: 2269.0334 - val_loss: 2205.9800
Epoch 7/1000
41/41 [==============================] - 0s 85us/step - loss: 2045.3301 - val_loss: 1267.9923
Epoch 8/1000
41/41 [==============================] - 0s 94us/step - loss: 2155.9314 - val_loss: 2341.7021
Epoch 9/1000
41/41 [==============================] - 0s 93us/step - loss: 2161.1418 - val_los

Epoch 76/1000
41/41 [==============================] - 0s 115us/step - loss: 2575.5942 - val_loss: 2683.3423
Epoch 77/1000
41/41 [==============================] - 0s 107us/step - loss: 2264.5212 - val_loss: 1300.4381
Epoch 78/1000
41/41 [==============================] - 0s 100us/step - loss: 2485.3508 - val_loss: 2401.2146
Epoch 79/1000
41/41 [==============================] - 0s 131us/step - loss: 2141.8665 - val_loss: 1180.4536
Epoch 80/1000
41/41 [==============================] - 0s 126us/step - loss: 2164.8103 - val_loss: 2330.9709
Epoch 81/1000
41/41 [==============================] - 0s 139us/step - loss: 2173.2041 - val_loss: 1390.1759
Epoch 82/1000
41/41 [==============================] - 0s 332us/step - loss: 2251.2478 - val_loss: 2207.4905
Epoch 83/1000
41/41 [==============================] - 0s 131us/step - loss: 2159.0513 - val_loss: 1536.8851
Epoch 84/1000
41/41 [==============================] - 0s 158us/step - loss: 2229.3701 - val_loss: 2096.8608
Epoch 85/1000
41/41

Epoch 151/1000
41/41 [==============================] - 0s 116us/step - loss: 2165.3416 - val_loss: 1273.0001
Epoch 152/1000
41/41 [==============================] - 0s 136us/step - loss: 2560.3401 - val_loss: 3115.5667
Epoch 153/1000
41/41 [==============================] - 0s 135us/step - loss: 2409.7859 - val_loss: 1302.7194
Epoch 154/1000
41/41 [==============================] - 0s 110us/step - loss: 2876.6089 - val_loss: 2908.6816
Epoch 155/1000
41/41 [==============================] - 0s 139us/step - loss: 2328.5381 - val_loss: 1186.0753
Epoch 156/1000
41/41 [==============================] - 0s 131us/step - loss: 2624.1633 - val_loss: 2755.1499
Epoch 157/1000
41/41 [==============================] - 0s 134us/step - loss: 2227.7981 - val_loss: 1093.7794
Epoch 158/1000
41/41 [==============================] - 0s 125us/step - loss: 2447.1853 - val_loss: 2640.4558
Epoch 159/1000
41/41 [==============================] - 0s 133us/step - loss: 2150.3179 - val_loss: 1053.0114
Epoch 160/

Epoch 226/1000
41/41 [==============================] - 0s 165us/step - loss: 2441.7141 - val_loss: 1435.9224
Epoch 227/1000
41/41 [==============================] - 0s 110us/step - loss: 2550.8684 - val_loss: 3265.9182
Epoch 228/1000
41/41 [==============================] - 0s 102us/step - loss: 2540.4214 - val_loss: 1424.0737
Epoch 229/1000
41/41 [==============================] - 0s 130us/step - loss: 2878.0811 - val_loss: 2865.8293
Epoch 230/1000
41/41 [==============================] - 0s 133us/step - loss: 2227.6411 - val_loss: 1032.7567
Epoch 231/1000
41/41 [==============================] - 0s 110us/step - loss: 2476.4080 - val_loss: 2741.1584
Epoch 232/1000
41/41 [==============================] - 0s 131us/step - loss: 2165.7869 - val_loss: 1007.3436
Epoch 233/1000
41/41 [==============================] - 0s 115us/step - loss: 2378.4319 - val_loss: 2624.0571
Epoch 234/1000
41/41 [==============================] - 0s 124us/step - loss: 2095.7922 - val_loss: 1023.4961
Epoch 235/

Epoch 301/1000
41/41 [==============================] - 0s 144us/step - loss: 2454.7539 - val_loss: 2603.7588
Epoch 302/1000
41/41 [==============================] - 0s 152us/step - loss: 2130.6704 - val_loss: 1136.3208
Epoch 303/1000
41/41 [==============================] - 0s 139us/step - loss: 2408.0254 - val_loss: 2599.2009
Epoch 304/1000
41/41 [==============================] - 0s 131us/step - loss: 2097.7964 - val_loss: 992.6829
Epoch 305/1000
41/41 [==============================] - 0s 162us/step - loss: 2335.8416 - val_loss: 2666.9736
Epoch 306/1000
41/41 [==============================] - 0s 155us/step - loss: 2130.1963 - val_loss: 949.1685
Epoch 307/1000
41/41 [==============================] - 0s 158us/step - loss: 2330.7957 - val_loss: 2757.1821
Epoch 308/1000
41/41 [==============================] - 0s 136us/step - loss: 2200.6616 - val_loss: 1194.3400
Epoch 309/1000
41/41 [==============================] - 0s 134us/step - loss: 2463.9915 - val_loss: 2652.8787
Epoch 310/10

...

Epoch 676/1000
41/41 [==============================] - 0s 108us/step - loss: 2197.1514 - val_loss: 2638.7009
Epoch 677/1000
41/41 [==============================] - 0s 84us/step - loss: 2154.0613 - val_loss: 1526.4821
Epoch 678/1000
41/41 [==============================] - 0s 214us/step - loss: 2566.9912 - val_loss: 2523.1687
Epoch 679/1000
41/41 [==============================] - 0s 81us/step - loss: 2073.6877 - val_loss: 1204.6519
Epoch 680/1000
41/41 [==============================] - 0s 115us/step - loss: 2267.0178 - val_loss: 2606.8491
Epoch 681/1000
41/41 [==============================] - 0s 161us/step - loss: 2039.3291 - val_loss: 907.1129
Epoch 682/1000
41/41 [==============================] - 0s 138us/step - loss: 2251.3398 - val_loss: 2787.8525
Epoch 683/1000
41/41 [==============================] - 0s 183us/step - loss: 2173.3076 - val_loss: 1021.8981
Epoch 684/1000
41/41 [==============================] - 0s 133us/step - loss: 2248.3801 - val_loss: 2363.0576
Epoch 685/100

Epoch 751/1000
41/41 [==============================] - 0s 137us/step - loss: 1997.7883 - val_loss: 983.3552
Epoch 752/1000
41/41 [==============================] - 0s 139us/step - loss: 2154.2686 - val_loss: 2696.5261
Epoch 753/1000
41/41 [==============================] - 0s 179us/step - loss: 2050.4470 - val_loss: 1046.7734
Epoch 754/1000
41/41 [==============================] - 0s 126us/step - loss: 2177.9939 - val_loss: 2551.6963
Epoch 755/1000
41/41 [==============================] - 0s 112us/step - loss: 1979.9718 - val_loss: 1024.0266
Epoch 756/1000
41/41 [==============================] - 0s 117us/step - loss: 2133.3865 - val_loss: 2663.0974
Epoch 757/1000
41/41 [==============================] - 0s 125us/step - loss: 2043.3948 - val_loss: 1052.7462
Epoch 758/1000
41/41 [==============================] - 0s 141us/step - loss: 2244.7070 - val_loss: 2560.5933
Epoch 759/1000
41/41 [==============================] - 0s 232us/step - loss: 2062.8276 - val_loss: 1119.4050
Epoch 760/1

Epoch 826/1000
41/41 [==============================] - 0s 142us/step - loss: 2291.7998 - val_loss: 2842.6531
Epoch 827/1000
41/41 [==============================] - 0s 113us/step - loss: 2110.4343 - val_loss: 799.9775
Epoch 828/1000
41/41 [==============================] - 0s 127us/step - loss: 2246.8142 - val_loss: 2735.4919
Epoch 829/1000
41/41 [==============================] - 0s 139us/step - loss: 2049.6128 - val_loss: 839.0489
Epoch 830/1000
41/41 [==============================] - 0s 131us/step - loss: 2222.5439 - val_loss: 2828.2803
Epoch 831/1000
41/41 [==============================] - 0s 113us/step - loss: 2161.3784 - val_loss: 1196.9739
Epoch 832/1000
41/41 [==============================] - 0s 129us/step - loss: 2444.5525 - val_loss: 2686.6047
Epoch 833/1000
41/41 [==============================] - 0s 94us/step - loss: 2138.8540 - val_loss: 1214.3094
Epoch 834/1000
41/41 [==============================] - 0s 144us/step - loss: 2291.9414 - val_loss: 2529.9812
Epoch 835/100

Epoch 901/1000
41/41 [==============================] - 0s 142us/step - loss: 2001.2820 - val_loss: 1018.1865
Epoch 902/1000
41/41 [==============================] - 0s 161us/step - loss: 2081.8210 - val_loss: 2684.5171
Epoch 903/1000
41/41 [==============================] - 0s 140us/step - loss: 2055.7292 - val_loss: 1112.7365
Epoch 904/1000
41/41 [==============================] - 0s 148us/step - loss: 2302.7522 - val_loss: 2645.1179
Epoch 905/1000
41/41 [==============================] - 0s 117us/step - loss: 2158.6199 - val_loss: 1171.2948
Epoch 906/1000
41/41 [==============================] - 0s 99us/step - loss: 2154.9604 - val_loss: 2496.3113
Epoch 907/1000
41/41 [==============================] - 0s 126us/step - loss: 2023.4785 - val_loss: 1203.8177
Epoch 908/1000
41/41 [==============================] - 0s 152us/step - loss: 2333.1731 - val_loss: 2776.1301
Epoch 909/1000
41/41 [==============================] - 0s 106us/step - loss: 2223.0742 - val_loss: 1249.2512
Epoch 910/1

Epoch 976/1000
41/41 [==============================] - 0s 122us/step - loss: 2093.2571 - val_loss: 2516.3525
Epoch 977/1000
41/41 [==============================] - 0s 127us/step - loss: 1930.3279 - val_loss: 972.5256
Epoch 978/1000
41/41 [==============================] - 0s 147us/step - loss: 2052.2891 - val_loss: 2527.4670
Epoch 979/1000
41/41 [==============================] - 0s 100us/step - loss: 1963.5748 - val_loss: 1073.2761
Epoch 980/1000
41/41 [==============================] - 0s 121us/step - loss: 2046.6635 - val_loss: 2549.4846
Epoch 981/1000
41/41 [==============================] - 0s 153us/step - loss: 2044.5989 - val_loss: 1374.4840
Epoch 982/1000
41/41 [==============================] - 0s 142us/step - loss: 2177.6516 - val_loss: 2298.3411
Epoch 983/1000
41/41 [==============================] - 0s 171us/step - loss: 1964.7107 - val_loss: 1393.0522
Epoch 984/1000
41/41 [==============================] - 0s 114us/step - loss: 2063.9709 - val_loss: 2233.7478
Epoch 985/1

## 產生 Predict 2019/04/02 ～ 2019/04/08 所需的Testing feature
取 2018/04/02 ～ 2018/04/08 以及網站上的 2019/04/02前一週的資料
但沒有2019/04/01所以用2018/04/01代替

In [39]:
def getTestX(tp_2018):
    findDate = 0
    index = 0
    test_list = []
    for d in tp_2018:
        if d[0] == '20180402':
            findDate = 1
            index += 1
            test_list += d[1:]
        if findDate is 1:
            index += 1
            test_list += d[1:]
            #test_list.append(d)
        if index is 7:
            break
    return test_list

In [40]:
testList = getTestX(tp_2018)
test_2019_list = ['28756', '1887', '6.56', '29140', '1933', '6.63', '30093', '1892', '6.29', '29673', '2054', '6.92', '25810', '2155', '8.35', '24466', '2298', '9.39', '23895', '1655', '6.92']
test_X = testList + test_2019_list
np_test_X = np.array(test_X)
#print(test_X)
#print(len(test_X))

In [41]:
#print(np_test_X.shape)
test = np_test_X.reshape(1, 42)
#print(test.shape)
test_Y = LinearModel.predict(test)
print(test_Y[0])

[28236.53  29439.281 27261.932 27617.697 27167.855 25013.686 23651.664]


In [42]:
def saveCSV(test_Y):
    date = 20190402
    d_list = []
    temp_list = []
    index = 0
    for t in test_Y:
        temp_list.append(str(date+index))
        temp_list.append(t)
        d_list.append(temp_list)
        temp_list = []
        index += 1
    df = pd.DataFrame(d_list, columns=['date', 'peak_load(MW)'])
    return df

In [43]:
df = saveCSV(test_Y[0])
df.to_csv('submission.csv', index=0)

2019.04.01
P76071200
資工所 馬崇堯